<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [747]:
#импорт всех библиотек
import pandas as pd
import psycopg2
# для задания 7
import requests
from bs4 import BeautifulSoup

In [748]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 

#не забудьте ввести параметры!!

In [749]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [750]:
# текст запроса
query_1 = f'''select count(*) num_vacancies
           from VACANCIES
'''

In [751]:
# результат запроса

num_vacancies = pd.read_sql_query(query_1, connection)
num_vacancies

C:\Users\User\AppData\Local\Temp\ipykernel_1016\765062197.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  num_vacancies = pd.read_sql_query(query_1, connection)


,num_vacancies
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [752]:
# текст запроса
query_2 = f'''select count(*) num_employers
           from employers
'''

In [753]:
# результат запроса

num_employers = pd.read_sql_query(query_2, connection)
num_employers

C:\Users\User\AppData\Local\Temp\ipykernel_1016\3232590113.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  num_employers = pd.read_sql_query(query_2, connection)


,num_employers
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [754]:
# текст запроса

query_3 = f'''select count(*) num_areas
           from areas
'''

In [755]:
# результат запроса

num_areas = pd.read_sql_query(query_3, connection)
num_areas

C:\Users\User\AppData\Local\Temp\ipykernel_1016\588818819.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  num_areas = pd.read_sql_query(query_3, connection)


,num_areas
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [756]:
# текст запроса

query_4 = f'''select count(*) num_industries
           from industries
'''

In [757]:
# результат запроса

num_industries = pd.read_sql_query(query_4, connection)
num_industries

C:\Users\User\AppData\Local\Temp\ipykernel_1016\3554130749.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  num_industries = pd.read_sql_query(query_4, connection)


,num_industries
0,294


***

# выводы по предварительному анализу данных

В рамках предварительного анализа данных была изучена общая структура базы данных, с которой будет проведена дальнейшая работа. 

Так база вакансий содержит информацию о 49 197 открытых вакансиях.
В базе содержится информация о 23 501 работодателях, которые заняты в 294 возможных сферах деятельности. База вакансий покрывает 1 362 различных региона.

Таким образом перед нами достаточно общирная база данных, анализ которой дает возможность выявить существующие закономерности и сделать выводы относительно состояния рынка вакансий.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [758]:
# текст запроса

query_5 = f'''select a.name area, count(v.*) cnt
           from VACANCIES v
           left join areas a on v.area_id = a.id
           group by a.name
           order by cnt desc
'''

In [759]:
# результат запроса

areas_vacancies = pd.read_sql_query(query_5, connection)
areas_vacancies.head(10)

C:\Users\User\AppData\Local\Temp\ipykernel_1016\342197131.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  areas_vacancies = pd.read_sql_query(query_5, connection)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [760]:
# текст запроса

query_6 = f'''select count(*) salary_exist
           from VACANCIES 
           where    salary_from is not null or 
                    salary_to is not null
'''           


In [761]:
# результат запроса

salary_exist = pd.read_sql_query(query_6, connection)
salary_exist

C:\Users\User\AppData\Local\Temp\ipykernel_1016\4259573840.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  salary_exist = pd.read_sql_query(query_6, connection)


,salary_exist
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [762]:
# текст запроса

query_7 = f'''select 
                round(avg(salary_from)) avg_min_salary, round(avg(salary_to)) avg_max_salary
            from VACANCIES
            ''' 

In [763]:
# результат запроса

avg_salary = pd.read_sql_query(query_7, connection)
avg_salary

C:\Users\User\AppData\Local\Temp\ipykernel_1016\69288754.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_salary = pd.read_sql_query(query_7, connection)


,avg_min_salary,avg_max_salary
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [764]:
# текст запроса

query_8 = f'''select schedule, employment, count(id) num_vacancies
           from VACANCIES 
           group by schedule, employment
           order by count(id) desc
           
'''

In [765]:
# результат запроса

type_regime_vacancies = pd.read_sql_query(query_8, connection)
type_regime_vacancies

C:\Users\User\AppData\Local\Temp\ipykernel_1016\2497275278.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  type_regime_vacancies = pd.read_sql_query(query_8, connection)


,schedule,employment,num_vacancies
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [766]:
# текст запроса

query_9 = f'''select experience, count(id)
           from VACANCIES 
           group by experience
           order by count(id) 
           
'''

In [767]:
# результат запроса

experience_vacancies = pd.read_sql_query(query_9, connection)
experience_vacancies

C:\Users\User\AppData\Local\Temp\ipykernel_1016\2900104077.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  experience_vacancies = pd.read_sql_query(query_9, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [768]:
# для выводов по детальному анализу данных

#доля вакансий, открытых в Москве от общего числа
top1_pers = areas_vacancies['cnt'].head(1)/areas_vacancies['cnt'].sum()*100
display(top1_pers)
#доля вакансий, открытых в топ 10 регионов от общего числа
top10_pers = areas_vacancies['cnt'].head(10).sum()/areas_vacancies['cnt'].sum()*100
display(top10_pers)

#кол-во регионов, где есть открытые вакансии
active_areas = areas_vacancies.count()
display(active_areas)

#доля вакансий с полной занятостью, полным днем от общего числа вакансий
regime_vac_pers = type_regime_vacancies['num_vacancies'].head(1)/ type_regime_vacancies['num_vacancies'].sum()*100
display(regime_vac_pers)

#вакансии с полной занятостью
full_time_vac = type_regime_vacancies[type_regime_vacancies['employment'] == 'Полная занятость']['num_vacancies'].sum()

#вакансии с полной занятостью и удаленным режимом работы
full_time_remote_vac = type_regime_vacancies[(type_regime_vacancies['employment'] == 'Полная занятость') & (type_regime_vacancies['schedule'] == 'Удаленная работа')]['num_vacancies'].sum()

#доля вакансий с полной занятостью и удаленным режимом работы от вакансий с полной занятостью
pers_remote = full_time_remote_vac/full_time_vac*100
display(pers_remote)

0    10.840092
Name: cnt, dtype: float64

43.543305486106874

area    769
cnt     769
dtype: int64

0    71.88853
Name: num_vacancies, dtype: float64

16.935466365668887

# выводы по детальному анализу вакансий

Только в 56.5% из общего числа регионов в базе (769 из 1 362) есть открытые вакансии. При этом из общего числа открытых вакансий - 10,8% (или 5 333 вакансии) приходятся на Москву, на топ-10 регионов приходится 43.5% из общего числа вакансий. Таким образом, обнаружена существенная концентрация рынка труда в Москве, с завидным отставанием остальных из топ-10 регионов (Санкт-Петербург, Минск, Новосибирск, Алматы, Екатеринбург, Нижний Новгород, Казань, Краснодар, Самара).

Почти половина вакансий в базе содержит сведения о заработной плате (24073 вакансии - 48,9% от общего числа). 71% (35 367) из общего числа открытых вакансий приходится на вакансии, которые подразумевают полную занятость на полный рабочий день. Соискатели, которые заинтересованы в удаленной работе на полный день имеют существенной меньшую возможность трудоустройства, тк таких вакансий всего лишь 7 802 из 46 069 вакансий, подразумевающих полную занятость - что составляет только 16.9% вакансий.

В основном компании размещают вакансии для специалистов с небольшим опытом (1-3 года) - 26 152 вакансий приходится именно на такой опыт работы. Меньше всего вакансий - 1 337 открыто для опытных соискателей (более 6 лет опыта). Что выглядит более чем логично, тк такие вакансии зачастую являются редкими, специфичными и высокооплачиваемыми, поэтому поиск сотрудников происходит во основном не через открытые базы и сервисы, а с помощью поиска конкретных рекомендаций в профессиональной среде

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [769]:
# текст запроса

query_10 = f'''select e.name, count(v.id) num_vacancies
           from VACANCIES v
           left join EMPLOYERS e on v.employer_id = e.id
           group by employer_id, e.name
           order by count(v.id) desc
           limit 5
           
'''

In [770]:
# результат запроса

employers_top = pd.read_sql_query(query_10, connection)
employers_top

C:\Users\User\AppData\Local\Temp\ipykernel_1016\588085447.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_top = pd.read_sql_query(query_10, connection)


,name,num_vacancies
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [771]:
# текст запроса

query_11 = f''' select a.id, a.name, vac.count_vac, emp.count_emp  
                    
                    from areas a

                    left join 
                    
                    (select 
                    v.area_id vac_area_id,
                    count(v.id) count_vac
                                from VACANCIES v 
                                group by v.area_id) vac          
                    on a.id = vac.vac_area_id 
                    
                    left join 
                    
                    (select 
                    e.area emp_area_id,
                    count(e.id) count_emp
                                from employers e 
                                group by e.area) emp          
                    on a.id = emp.emp_area_id
                    
                    where  emp.count_emp is not null
                    order by vac.count_vac desc, emp.count_emp desc
                    
           
'''

In [772]:
# результат запроса

areas_vac_empl = pd.read_sql_query(query_11, connection)
areas_vac_empl

C:\Users\User\AppData\Local\Temp\ipykernel_1016\443002372.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  areas_vac_empl = pd.read_sql_query(query_11, connection)


,id,name,count_vac,count_emp
0,113,Россия,NaN,410
1,40,Казахстан,NaN,207
2,2019,Московская область,NaN,75
3,1438,Краснодарский край,NaN,19
4,1530,Ростовская область,NaN,18
...,...,...,...,...
649,2048,Электроугли,1.0,1
650,1443,Белореченск,1.0,1
651,1542,Красный Сулин,1.0,1
652,1689,Заволжье,1.0,1


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [773]:
# текст запроса

query_12 = f'''select e.name, count(distinct v.area_id) num_area
           from VACANCIES v
           left join EMPLOYERS e on v.employer_id = e.id
           group by employer_id, e.name
           order by count(distinct v.area_id) desc
           
'''

In [774]:
# результат запроса

employers_in_area = pd.read_sql_query(query_12, connection)
employers_in_area

C:\Users\User\AppData\Local\Temp\ipykernel_1016\3563068010.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_in_area = pd.read_sql_query(query_12, connection)


,name,num_area
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [775]:
# текст запроса

query_13 = f'''select count(emp.id) employers_no_indust
           from EMPLOYERS emp
           left join EMPLOYERS_INDUSTRIES ind on emp.id = ind.employer_id
           where  ind.industry_id is null
'''

In [776]:
# результат запроса

employers_no_indust = pd.read_sql_query(query_13, connection)
employers_no_indust

C:\Users\User\AppData\Local\Temp\ipykernel_1016\2276689688.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_no_indust = pd.read_sql_query(query_13, connection)


,employers_no_indust
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [777]:
# текст запроса

query_14 = f'''select emp.name
           from EMPLOYERS emp
           left join EMPLOYERS_INDUSTRIES ind on emp.id = ind.employer_id
           where  ind.industry_id is not null
           group by emp.name
           having count(ind.industry_id) = 4
           order by emp.name
           offset 2 limit 1
'''

In [778]:
# результат запроса

employers_four_indust = pd.read_sql_query(query_14, connection)
employers_four_indust

C:\Users\User\AppData\Local\Temp\ipykernel_1016\2866917526.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_four_indust = pd.read_sql_query(query_14, connection)


,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [779]:
# текст запроса

query_15 = f'''select count(emp.id)
           from EMPLOYERS emp
           left join EMPLOYERS_INDUSTRIES ind on emp.id = ind.employer_id
           left join INDUSTRIES i on i.id = ind.industry_id
           where i.name  like 'Разработка программного обеспечения'

'''

In [780]:
# результат запроса

employers_po = pd.read_sql_query(query_15, connection)
employers_po

C:\Users\User\AppData\Local\Temp\ipykernel_1016\2221595794.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_po = pd.read_sql_query(query_15, connection)


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [781]:
# код для получения списка городов-милионников

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Определяем адрес страницы
response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
text = BeautifulSoup(response.text, "html.parser")
table = text.select_one("table.standard")

mil_cities = []
 
for row in table.find_all("tr"):
    cells = row.find_all("td")
     
    if len(cells) > 0:
        city = cells[1].text.strip()
        mil_cities.append(city)
        
df_mil_cities = pd.DataFrame(mil_cities,columns =['name'])
df_mil_cities

,name
0,Москва
1,Санкт-Петербург
2,Новосибирск
3,Екатеринбург
4,Казань
5,Нижний Новгород
6,Красноярск
7,Челябинск
8,Самара
9,Уфа


In [782]:
# текст запроса

# текст запроса
query_16 = f'''select  a.name, count (v.id)
           from VACANCIES v
           left join EMPLOYERS e on v.employer_id = e.id
           left join AREAS a on a.id = v.area_id
           where e.name like 'Яндекс'
           group by   a.name
           order by count (v.id) desc
'''

In [783]:
# результат запроса

df_yandex_cities = pd.read_sql_query(query_16, connection)

yandex_mil_cities = df_mil_cities.merge(
    df_yandex_cities ,
    on='name',
    how='left'
)

yandex_mil_cities = yandex_mil_cities.sort_values(by='count')
yandex_mil_cities.loc[ len(yandex_mil_cities.index )] = ['Total', yandex_mil_cities['count'].sum()]
yandex_mil_cities

C:\Users\User\AppData\Local\Temp\ipykernel_1016\805429357.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_yandex_cities = pd.read_sql_query(query_16, connection)


,name,count
12,Омск,21
7,Челябинск,22
6,Красноярск,23
15,Волгоград,24
4,Казань,25
10,Ростов-на-Дону,25
14,Пермь,25
8,Самара,26
9,Уфа,26
11,Краснодар,30


***

In [784]:
# для выводов по анализу работодателей

#доля работодателей без описания сферы деятельности
pers_emp_no_indust = employers_no_indust['employers_no_indust']/num_employers['num_employers']*100
display(pers_emp_no_indust)

0    35.824007
dtype: float64

# выводы по анализу работодателей

В рамках анализа работодателей обратим внимание на следующие аспекты: компания Яндекс является топ-1 работодателей по числу вакансий (1 933 шт) и охвату территорий, где представлены вакансии (181 регион). Из прочих крупных компаний с большим кол-вом вакансий (Ростелеком, Тинькофф, Сбер, Газпром нефть), только Ростелеком имеет существенный охват регионов - 151 регион, остальные в список не попали, что указывает на специфичность каждой отрасли, так Банки (Тинькофф и Сбер), а также добывающие компании (Газпром нефть) ввиду своих особенностей имеют более высокую  концентрацию в регионе размещения головного офиса/производства.

25% от всех вакансий компании Яндекс (485) опубликованы в городах-миллионниках.Где, ожидаемо, лидирует Москва и Санкт-Петербург, однако концентрация вакансий меньше, чем можно было бы ожидать.
Минимальное количество вакансий Яндекса  в городах-миллионниках в Омске (21 шт) - максимальное в Москве (54шт).

35% от всех работодателей не внесли информацию о сфере их деятельности (8 419 компании), что скорее всего будет снижает уровень осведомленности и доверия к вакансиям таких компаний для кандидатов. Особенно  актуально это для некрупных компаний, которые "не на слуху".  


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [785]:
# текст запроса

query_17 = f'''select count(id) num_data_vacancies
           from vacancies 
           WHERE LOWER(NAME) like '%данн%' or LOWER(NAME)  like '%data%' 
           

'''

In [786]:
# результат запроса

data_vacancies = pd.read_sql_query(query_17, connection)
data_vacancies

C:\Users\User\AppData\Local\Temp\ipykernel_1016\656470052.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_vacancies = pd.read_sql_query(query_17, connection)


,num_data_vacancies
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [787]:
# текст запроса

query_18 = f'''select count(id) num_jun_ds
           from vacancies 
           WHERE 
           
           (
               LOWER(NAME) like '%data scientist%' or 
                LOWER(NAME)  like '%data science%'  or 
                LOWER(NAME)  like '%исследователь данных%' or
                (NAME  like '%ML%' AND name not like '%HTML%') or
                LOWER(NAME) like '%machine learning%' or
                LOWER(NAME)  like '%машинн%обучен%' 

            )
            and
           
           (EMPLOYMENT = 'Стажировка' or 
               LOWER(NAME)  like '%junior%' or 
               EXPERIENCE = 'Нет опыта'
               )
'''

In [788]:
# результат запроса

junior_ds_vacancies = pd.read_sql_query(query_18, connection)
junior_ds_vacancies

C:\Users\User\AppData\Local\Temp\ipykernel_1016\3656829797.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  junior_ds_vacancies = pd.read_sql_query(query_18, connection)


,num_jun_ds
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [789]:
# текст запроса

query_19 = f'''select count(*) num_ds_sql_vac
           from vacancies 
           WHERE 
           
           (
               LOWER(NAME) like '%data scientist%' or 
                LOWER(NAME)  like '%data science%'  or 
                LOWER(NAME)  like '%исследователь данных%' or
                (NAME  like '%ML%' AND name not like '%HTML%') or
                LOWER(NAME) like '%machine learning%' or
                LOWER(NAME)  like '%машинн%обучен%' 

            )
            
            and
            
            (lower(key_skills) like '%sql%' or lower(key_skills) like '%postgres%')
            

'''

In [790]:
# результат запроса

ds_sql_vacancies = pd.read_sql_query(query_19, connection)
ds_sql_vacancies

C:\Users\User\AppData\Local\Temp\ipykernel_1016\684682554.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ds_sql_vacancies = pd.read_sql_query(query_19, connection)


,num_ds_sql_vac
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [791]:
# текст запроса

query_20 = f'''select count(*) num_ds_pyth_vac
           from vacancies 
           WHERE 
           
           (
               LOWER(NAME) like '%data scientist%' or 
                LOWER(NAME)  like '%data science%'  or 
                LOWER(NAME)  like '%исследователь данных%' or
                (NAME  like '%ML%' AND name not like '%HTML%') or
                LOWER(NAME) like '%machine learning%' or
                LOWER(NAME)  like '%машинн%обучен%' 

            )
            
            and
            
            (lower(key_skills) like '%python%' or lower(key_skills) like '%питон%')
            
  
'''

In [792]:
# результат запроса

ds_python_vacancies = pd.read_sql_query(query_20, connection)
ds_python_vacancies

C:\Users\User\AppData\Local\Temp\ipykernel_1016\2031589014.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ds_python_vacancies = pd.read_sql_query(query_20, connection)


,num_ds_pyth_vac
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [793]:
# текст запроса

query_21= f'''SELECT ROUND(AVG(length(key_skills) - length(replace(key_skills, '\t', '')) + 1), 2) avg_skills
           from vacancies 
           WHERE 
           
           (
               LOWER(NAME) like '%data scientist%' or 
                LOWER(NAME)  like '%data science%'  or 
                LOWER(NAME)  like '%исследователь данных%' or
                (NAME  like '%ML%' AND name not like '%HTML%') or
                LOWER(NAME) like '%machine learning%' or
                LOWER(NAME)  like '%машинн%обучен%' 

            )
  
'''


In [794]:
# результат запроса

avg_skills_ds = pd.read_sql_query(query_21, connection)
avg_skills_ds

C:\Users\User\AppData\Local\Temp\ipykernel_1016\438277652.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_skills_ds = pd.read_sql_query(query_21, connection)


,avg_skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [795]:
# текст запроса

query_22 = f'''SELECT experience, 
                round(avg((COALESCE(salary_from,salary_to) + COALESCE(salary_to,salary_from))/2),0) as avg_salary
           from vacancies 
           WHERE 
           
           (
               LOWER(NAME) like '%data scientist%' or 
                LOWER(NAME)  like '%data science%'  or 
                LOWER(NAME)  like '%исследователь данных%' or
                (NAME  like '%ML%' AND name not like '%HTML%') or
                LOWER(NAME) like '%machine learning%' or
                LOWER(NAME)  like '%машинн%обучен%' 

            )
            and
            (
                salary_from is not null
                or
                salary_to	is not null
                )
            group by   experience
            order by experience       
 
'''

In [796]:
# результат запроса


avg_salary = pd.read_sql_query(query_22, connection)
avg_salary

C:\Users\User\AppData\Local\Temp\ipykernel_1016\1715107739.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_salary = pd.read_sql_query(query_22, connection)


,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

In [797]:
# для выводов по предметному анализу

# доля вакансий про данные от общего числа вакансий
pers_data_vac = data_vacancies['num_data_vacancies']/num_vacancies['num_vacancies']*100
display(pers_data_vac)

#все дата саейнс вакансии

query_x = f'''select count(id) num_ds
           from vacancies 
           WHERE 
           
           (
               LOWER(NAME) like '%data scientist%' or 
                LOWER(NAME)  like '%data science%'  or 
                LOWER(NAME)  like '%исследователь данных%' or
                (NAME  like '%ML%' AND name not like '%HTML%') or
                LOWER(NAME) like '%machine learning%' or
                LOWER(NAME)  like '%машинн%обучен%' 

            )
'''

ds_vacancies = pd.read_sql_query(query_x, connection)
display(ds_vacancies)
# доля дата сайенс вакансий от общего числа вакансий
pers_ds_vac = ds_vacancies['num_ds']/num_vacancies['num_vacancies']*100
display(pers_ds_vac)

0    3.599813
dtype: float64

C:\Users\User\AppData\Local\Temp\ipykernel_1016\2120015915.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ds_vacancies = pd.read_sql_query(query_x, connection)


,num_ds
0,480


0    0.975669
dtype: float64

In [798]:
# для выводов по предметному анализу
# ключевые работодатели для дата саентистов

query_xx = f'''select e.name, count(v.id)
           from vacancies v
           left join  EMPLOYERS  e on v.employer_id = e.id
           WHERE 
           
           (
               LOWER(v.NAME) like '%data scientist%' or 
                LOWER(v.NAME)  like '%data science%'  or 
                LOWER(v.NAME)  like '%исследователь данных%' or
                (v.NAME  like '%ML%' AND v.name not like '%HTML%') or
                LOWER(v.NAME) like '%machine learning%' or
                LOWER(v.NAME)  like '%машинн%обучен%' 

            )
            group by e.name
            order by count(v.id) desc
            limit 10
            

'''
ds_top_employers = pd.read_sql_query(query_xx, connection)
ds_top_employers

C:\Users\User\AppData\Local\Temp\ipykernel_1016\2655710487.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ds_top_employers = pd.read_sql_query(query_xx, connection)


,name,count
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
5,Яндекс,9
6,EvenBet Gaming,9
7,МегаФон,8
8,Andersen,7
9,Ozon,6


In [799]:
# для выводов по предметному анализу
#средняя зп для всех вакансий
query_xxx = f'''SELECT experience, 
                round(avg((COALESCE(salary_from,salary_to) + COALESCE(salary_to,salary_from))/2),0) as avg_salary
           from vacancies 
           WHERE 

            (
                salary_from is not null
                or
                salary_to	is not null
                )
            group by   experience
            order by experience       
 
'''
avg_salary_full = pd.read_sql_query(query_xxx, connection)
display(avg_salary_full)

#сравним разницу в ЗП
avg_salary_comparison = avg_salary_full.merge(avg_salary, how = 'left', left_on='experience', right_on='experience', suffixes=('_x', '_y'),indicator=False,)
avg_salary_comparison['delta'] = (avg_salary_comparison['avg_salary_y']- avg_salary_comparison['avg_salary_x'])/avg_salary_comparison['avg_salary_x']*100
avg_salary_comparison


C:\Users\User\AppData\Local\Temp\ipykernel_1016\1063762291.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_salary_full = pd.read_sql_query(query_xxx, connection)


,experience,avg_salary
0,Более 6 лет,166256.0
1,Нет опыта,40472.0
2,От 1 года до 3 лет,76541.0
3,От 3 до 6 лет,133211.0


,experience,avg_salary_x,avg_salary_y,delta
0,Более 6 лет,166256.0,NaN,NaN
1,Нет опыта,40472.0,74643.0,84.431212
2,От 1 года до 3 лет,76541.0,139675.0,82.483898
3,От 3 до 6 лет,133211.0,243115.0,82.503697


Доля вакансий, которые имеют отношение к данным крайне мала в общем объеме - это 3.6% (или 1 771 шт). Вакансий для дата сайентистов еще меньше, менее 1% (480 шт), вакансий без опыта работы в этом направлении - лишь 51 шт. Таким образом, не смотря на высокий интерес к дата сайенс как со стороны работодателей, так и со стороны соискателей рынок труда по данной специализации все еще мал, тк таких специалистов могут позволить себе только крупные компании, готовые инвестировать в развитие таких направлений деятельности, топ 5 из которых это Сбер - 37 вакансий, Bell Integrator - 25 вакансий, ВТБ - 18, VK-15, Positive Technologies - 11 (Яндекс занимает 5 место и 9 ваканий).

При этом стоит обратить внимание на ключевые навыки, которыми должен обладать соискатель, заинтересованные в трудоустройстве в качестве дата саентиста: знание Python указано в 351 вакансии, SQL - в 201 вакансии.

Однако, несмотря на тот факт, что вакансий для дата саентистов мало на рынке, все предложенные вакансии с информацией о заработной плате имеют среднюю зп почти в 2 раза выше (82-84% в зависимости от опыта) чем, средняя зп по всем вакансиям, что объясняет привлекательность данной сферы для соискателей.

# Общий вывод по проекту

In [800]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования


#средняя зп для всех вакансий в зависимости от региона
query_xxx = f'''SELECT a.name , count(v.id),
                round(avg((COALESCE(v.salary_from,v.salary_to) + COALESCE(v.salary_to,v.salary_from))/2),0) as avg_salary
           from vacancies v
           left join AREAS a on a.id = v.area_id
           WHERE 

            (
                v.salary_from is not null
                or
                v.salary_to	is not null
                )
            group by   a.name
            order by avg_salary desc       
 
'''
avg_salary_full_area = pd.read_sql_query(query_xxx, connection)

#посмотрим макс средние зп в базе
display(avg_salary_full_area.head(10))

#средняя зп для городов милионников
df_mil_cities1 = df_mil_cities

mil_cities_avg_salary = df_mil_cities1.merge(
    avg_salary_full_area ,
    on='name',
    how='left'
)


display(mil_cities_avg_salary)

C:\Users\User\AppData\Local\Temp\ipykernel_1016\76327812.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_salary_full_area = pd.read_sql_query(query_xxx, connection)


,name,count,avg_salary
0,Германия,2,452751.0
1,Болгария,3,324348.0
2,Испания,1,300000.0
3,Черногория,4,288589.0
4,Литва,2,263019.0
5,Польша,3,257156.0
6,Петропавловка (Республика Бурятия),1,250000.0
7,Турция,19,235435.0
8,Кипр,39,223736.0
9,Кутаиси,3,220475.0


,name,count,avg_salary
0,Москва,1592,134140.0
1,Санкт-Петербург,1203,104410.0
2,Новосибирск,929,107777.0
3,Екатеринбург,901,91896.0
4,Казань,628,92207.0
5,Нижний Новгород,680,96059.0
6,Красноярск,500,75156.0
7,Челябинск,501,77463.0
8,Самара,505,80223.0
9,Уфа,383,74733.0


In [804]:
# для выводов общих
# режимы работы для дата саентистов

query_xxxx = f'''select schedule, employment, count(id) num_vacancies
           from vacancies v
           
           WHERE 
           
           (
               LOWER(v.NAME) like '%data scientist%' or 
                LOWER(v.NAME)  like '%data science%'  or 
                LOWER(v.NAME)  like '%исследователь данных%' or
                (v.NAME  like '%ML%' AND v.name not like '%HTML%') or
                LOWER(v.NAME) like '%machine learning%' or
                LOWER(v.NAME)  like '%машинн%обучен%' 

            )
                
            group by schedule, employment
            order by count(v.id) desc
            
        
'''
regime_ds = pd.read_sql_query(query_xxxx, connection)
display(regime_ds)

regime_ds['num_vacancies'].sum()

C:\Users\User\AppData\Local\Temp\ipykernel_1016\1470388572.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  regime_ds = pd.read_sql_query(query_xxxx, connection)


,schedule,employment,num_vacancies
0,Полный день,Полная занятость,319
1,Удаленная работа,Полная занятость,110
2,Гибкий график,Полная занятость,37
3,Гибкий график,Частичная занятость,4
4,Удаленная работа,Частичная занятость,3
5,Полный день,Стажировка,2
6,Удаленная работа,Проектная работа,2
7,Гибкий график,Стажировка,2
8,Полный день,Частичная занятость,1


480

Для исследования предоставлена общирная база данных,которая содержит сведения о 49 197 открытых вакансиях, 23 501 работодателях, которые заняты в 294 возможных сферах деятельности.  База вакансий покрывает 1 362 различных региона. 

При этом 56.5% из общего числа регионов в базе (769 из 1 362) имеют открытые вакансии. При этом из общего числа открытых вакансий - 10,8% (или 5 333 вакансии) приходятся на Москву, на топ-10 регионов приходится 43.5% из общего числа вакансий. Таким образом, можно судить у высокой положительной корреляции количества открытых вакансий и размером городов, и таким образом существенной
концентрации рынка труда в Москве и с существенным отставанием других крупных городах.

Большинство вакансий, находящихся в базе данных для специалистов с небольшим опытом (1-3 года) - 26 152 вакансий приходится именно на такой опыт работы. Меньше всего вакансий - 1 337 открыто для опытных соискателей (более 6 лет опыта). Средние заработные платы варьируются от 40 тыс руб для вакансий без опыта работы, и 166 тыс рублей для вакансий с опытом более 6 лет.

Почти половина вакансий в базе содержит сведения о заработной плате (24073 вакансии - 48,9% от общего числа). При этом также как и по количеству вакансий Москва - является лидером и по средней заработной плате (134 тыс руб) Санкт-Петербург и Новосибирск уступают Москве в средней заработной плате порядке 30 тыс рублей. Однако, максимальные средние зп предлагаются по вакансиям из Евросоюза (Германии - 453 тыс руб, Болгария - 324 тыс руб, Испания - 300 тыс руб), но таких вакансий - единицы - на топ 5 стран приходится 12 вакансий. 

Так что для дальнейшего анализа  имеет смысл рассматривать такие наблюдения как выбросы и очистить от них выборку и сфокусироваться только на регионах России или может быть России и странах СНГ.

По итогам проведенного анализа вакансий для специалистов в дата саейнс можно сделать следующие выводы: это высокооплачиваемая (зп почти в 2 раза выше средних по рынку), но достаточно редкая 
специализация, характерная для крупных IT компаний, Банков (финансового сектора), Телеком компаний. При этом работодатели заинтересованы в кандидатах со средним уровнем опыта, вакансий для junior позиций данного направления мало. Дата саентисты могут рассчитывать на более гибкий режим работы 22% от общего кол-ва вакансий предлагают удаленный режим работы,однако "стандартный режим работы" все же остается более предпочтительным. 


In [805]:
connection.close()